# CNN + Fine Tuning + Augmentation

The plan: Fine tune pre-trained model to transfer it to predict cat/dogs

In [2]:
from keras.applications import VGG16
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


Load VGG model without the last classifier layer, only the convolutional layers. 

Last layer has shape 7 x 7 x 512

In [3]:
image_size = 224
vgg_conv = VGG16(weights='imagenet',
                include_top=False,
                input_shape=(image_size, image_size, 3))

**For fine tuning, we only train the last convolutional layer of the model, and freeze the rest**

In [4]:
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False

for layer in vgg_conv.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x7f23f71b3908> False
<keras.layers.convolutional.Conv2D object at 0x7f23f713e710> False
<keras.layers.convolutional.Conv2D object at 0x7f23f713e588> False
<keras.layers.pooling.MaxPooling2D object at 0x7f23f7165198> False
<keras.layers.convolutional.Conv2D object at 0x7f23f71651d0> False
<keras.layers.convolutional.Conv2D object at 0x7f23f7116cf8> False
<keras.layers.pooling.MaxPooling2D object at 0x7f23f70c8160> False
<keras.layers.convolutional.Conv2D object at 0x7f23f70c8048> False
<keras.layers.convolutional.Conv2D object at 0x7f23f70e4470> False
<keras.layers.convolutional.Conv2D object at 0x7f23f7087748> False
<keras.layers.pooling.MaxPooling2D object at 0x7f23f703e080> False
<keras.layers.convolutional.Conv2D object at 0x7f23f703e0b8> False
<keras.layers.convolutional.Conv2D object at 0x7f23f705b4e0> False
<keras.layers.convolutional.Conv2D object at 0x7f23f65c20b8> False
<keras.layers.pooling.MaxPooling2D object at 0x7f23f65fb128>

**Add classifier on top of convolutional base, and build our classifier**

In [5]:
from keras import models, layers, optimizers

# Create model
model = models.Sequential()

# Add vgg convolutional base model
model.add(vgg_conv)

# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5)) # Drop out to avoid overfitting
model.add(layers.Dense(2, activation='softmax'))

# Print summary of model
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 2050      
Total params: 40,407,874
Trainable params: 32,772,610
Non-trainable params: 7,635,264
_________________________________________________________________


Use ImageDataGenerator class to load the images and flow_from_directory function to generate batches of images and labels

In [6]:
train_dir = '../data/train'
validation_dir = '../data/validation'
nTrain = 20000
nVal = 5000

In [7]:
# Perform data augmentation with width shift, height shift, rotation and horizontal flips
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

# Change the 
train_batchsize = 80
val_batchsize = 20

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(image_size, image_size),
    batch_size=train_batchsize,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(image_size, image_size),
    batch_size=val_batchsize,
    class_mode='categorical',
    shuffle=False
)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [ ]:
# Compile model
model.compile(loss='categorical_crossentropy',
             optimizer=optimizers.RMSprop(lr=1e-4),
             metrics=['acc'])

# Train the model
history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples/train_generator.batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples/validation_generator.batch_size,
    verbose=1
)

# Save the model
model.save('../output/vgg16_d1.h5')

Epoch 1/50
250/250 [==============================] - 13483s 54s/step - loss: 0.4235 - acc: 0.8283 - val_loss: 0.1736 - val_acc: 0.9288
Epoch 2/50
250/250 [==============================] - 13501s 54s/step - loss: 0.2540 - acc: 0.9019 - val_loss: 0.2086 - val_acc: 0.9036
Epoch 3/50
250/250 [==============================] - 13609s 54s/step - loss: 0.2431 - acc: 0.9128 - val_loss: 0.1417 - val_acc: 0.9478
Epoch 4/50
  5/250 [..............................] - ETA: 3:01:09 - loss: 0.1298 - acc: 0.9500